In [1]:
#!/usr/bin/env python
import sys
import copy
#Opencv
import cv2 as cv
#from cv_bridge import CvBridge, CvBridgeError
import numpy as np
from numpy import mat
# ROS packages
import rospy
import tf
import time
import logging
#Robotiq package
from robotiq_ft_sensor.msg import ft_sensor
from robotiq_2f_gripper_control.msg import _Robotiq2FGripper_robot_output as outputMsg
from robotiq_2f_gripper_control.msg import _Robotiq2FGripper_robot_input  as inputMsg
from robotiq_ft_sensor.msg import ft_sensor
# from robotiq_c_model_control.msg import _CModel_robot_output as outputMsg
# ROS Image message
import geometry_msgs.msg
from std_msgs.msg import Int32
from geometry_msgs.msg import PoseStamped, Pose
from geometry_msgs.msg import PoseWithCovariance
from geometry_msgs.msg import WrenchStamped
from sensor_msgs.msg import Image
from std_msgs.msg import String, Empty, UInt16
#Math
import math
from math import pi, sin, cos, atan2
from math import sqrt, pi, acos, sin, cos
import math3d as m3d
#UR Control
import urx
from apriltag_ros.msg import AprilTagDetection, AprilTagDetectionArray

from tf.transformations import euler_from_quaternion, quaternion_from_euler
from a4_paper_turning.msg import pos
import actionlib
import xlwt

from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from geometry_msgs.msg import WrenchStamped

In [2]:
#Connect to gripper
# gripper_pub = rospy.Publisher('Robotiq2FGripperRobotOutput', outputMsg.Robotiq2FGripper_robot_output, queue_size=10)
# arduino_pub1 = rospy.Publisher('/dac1', UInt16, queue_size=1)
# rospy.sleep(0.5)
# arduino_pub2 = rospy.Publisher('/dac2', UInt16, queue_size=1)
# rospy.sleep(0.5)
#Connect to UR robot
#robHong = urx.Robot("192.168.1.102")
arduino_pub1 = rospy.Publisher('/dac1', UInt16, queue_size=1)
arduino_pub2 = rospy.Publisher('/dac2', UInt16, queue_size=1)
servo_pub = rospy.Publisher('/servo', UInt16, queue_size=10)
rospy.sleep(0.5)
robKong = urx.Robot('192.168.1.10')
# robHong = urx.Robot('192.168.1.102')
# rospy.init_node('sub_pub',anonymous=True)
rospy.init_node('picking_acrylic_board', anonymous=True)
logging.basicConfig(level=logging.WARN)
# action_name = 'command_robotiq_action'
# robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
# robotiq_client.wait_for_server()
rospy.sleep(0.5)
listener = tf.TransformListener()

In [3]:
def robKong_go_to_home():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*90.16/180
    Kong_joint1 = -pi*70.08/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*130.76/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*150.66/180
    Kong_joint4 = -pi*89.97/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*0.17/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robKong_go_to_home1():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*78.65/180
    Kong_joint1 = -pi*68.25/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*118.22/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*140.31/180
    Kong_joint4 = -pi*90.83/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*84.31/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robKong_go_to_home_mid():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = pi*80/180
    Kong_joint1 = -pi*78.88/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*126.77/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*137.66/180
    Kong_joint4 = -pi*89.63/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*55/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robKong_go_to_corner1():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = pi*60.1/180
    Kong_joint1 = -pi*53.65/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*92.94/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*128.92/180
    Kong_joint4 = -pi*89.97/180#pi*1/2#-pi*1/2
    Kong_joint5 = pi*99.39/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robHong_go_to_home_mid():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*65.47/180
    Kong_joint1 = -pi*106.1/180#-pi*68.03/180 #81.05
    Kong_joint2 = -pi*103.22/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*61.23/180
    Kong_joint4 = pi*88.19/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*63.4/180
    robHong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robHong_go_to_home_temp():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*72.99/180
    Kong_joint1 = -pi*115.23/180#-pi*68.03/180 #81.05
    Kong_joint2 = -pi*90.02/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*65.15/180
    Kong_joint4 = pi*87.96/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*160.96/180
    robHong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)

In [4]:
def move_ref_tcp(target_ref_transform, tcp_to_frame, acc=0.1, vel=0.02, wait=True):
    temp1 = m3d.Transform.get_inverse(m3d.Transform(tcp_to_frame))
    #print temp1
    temp2 = m3d.Transform(target_ref_transform)
    delta_rotation = temp1.orient.inverse * temp2.orient * temp1.orient
    delta_translation = temp2.pos + temp2.orient*temp1.pos - temp1.pos
    Delta_tcp = m3d.Transform()
    Delta_tcp.orient = delta_rotation
    Delta_tcp.pos = temp1.orient.inverse * delta_translation
    #print m3d.Transform(Delta_tcp)
    robKong.movel_tool(m3d.Transform.get_pose_vector(Delta_tcp), acc, vel, wait)
def move_ref_tcp_hong(target_ref_transform, tcp_to_frame, acc=0.1, vel=0.02, wait=True):
    temp1 = m3d.Transform.get_inverse(m3d.Transform(tcp_to_frame))
    #print temp1
    temp2 = m3d.Transform(target_ref_transform)
    delta_rotation = temp1.orient.inverse * temp2.orient * temp1.orient
    delta_translation = temp2.pos + temp2.orient*temp1.pos - temp1.pos
    Delta_tcp = m3d.Transform()
    Delta_tcp.orient = delta_rotation
    Delta_tcp.pos = temp1.orient.inverse * delta_translation
    #print m3d.Transform(Delta_tcp)
    robHong.movel_tool(m3d.Transform.get_pose_vector(Delta_tcp), acc, vel, wait)

In [5]:
robKong.set_tcp((0,0,0,0,0,0))
rospy.sleep(1)
tag_pose_x1 = 0
tag_pose_y1 = 0
tag_angle1 = 0
is_send_tag1=0
tag_topic1 = '/tag_detections'
#Robot motion speed percentage (0.5 ~ 3.0)

def kong_approach_the_object(a):

    global is_send_tag1
    is_send_tag1=int(a)    
    
def kong_detect_the_object(a):
    
    global is_send_tag1
    is_send_tag1=int(a)       
    
def kong_tag_callback(msg):
    global is_send_tag1
    global tag_pose_x1
    global tag_pose_y1
    global tag_pose_z1
    global tag_angle1

    if is_send_tag1 == 1:
        tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation1.x, tag_orientation1.y, tag_orientation1.z, tag_orientation1.w])
        tag_angle1 = rpy[2]
        tag_pose_x1 = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y1 = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z1 = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x1, ',tag_y = ', tag_pose_y1, ',tag_z- ',tag_pose_z1, ',tag angle = ', tag_angle1/ 3.14159296 * 180
        rospy.sleep(1)
        robKong.translate_tool((-tag_pose_x1, -tag_pose_y1, 0), acc=0.1, vel=0.1, wait=True)
        is_send_tag1=0

    if is_send_tag1 == 2:
        tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation1.x, tag_orientation1.y, tag_orientation1.z, tag_orientation1.w])
        tag_angle1 = rpy[2]
        tag_pose_x1 = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y1 = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z1 = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x1, ',tag_y = ', tag_pose_y1, ',tag_z- ',tag_pose_z1, ',tag angle = ', tag_angle1/ 3.14159296 * 180
        is_send_tag1=0

In [6]:
rospy.sleep(1)
tag_pose_x = 0
tag_pose_y = 0
tag_angle = 0
is_send_tag=0
tag_topic2 = '/cam2/tag_detections'
#Robot motion speed percentage (0.5 ~ 3.0)

def hong_approach_the_object(a):

    global is_send_tag
    is_send_tag=int(a)   
    
def hong_detect_the_object(a):
    
    global is_send_tag
    is_send_tag=int(a)
    
def hong_tag_callback(msg):
    global is_send_tag
    global tag_pose_x
    global tag_pose_y
    global tag_pose_z
    global tag_angle

    if is_send_tag == 1:
        tag_orientation = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation.x, tag_orientation.y, tag_orientation.z, tag_orientation.w])
        tag_angle = rpy[2]
        tag_pose_x = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x, ',tag_y = ', tag_pose_y, ',tag_z- ',tag_pose_z, ',tag angle = ', tag_angle/ 3.14159296 * 180
        rospy.sleep(1)
        robHong.translate_tool((tag_pose_y, -tag_pose_x, 0), acc=0.1, vel=0.1, wait=True)
        is_send_tag=0

    if is_send_tag == 2:
        tag_orientation = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation.x, tag_orientation.y, tag_orientation.z, tag_orientation.w])
        tag_angle = rpy[2]
        tag_pose_x = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x, ',tag_y = ', tag_pose_y, ',tag_z- ',tag_pose_z, ',tag angle = ', tag_angle/ 3.14159296 * 180
        is_send_tag=0

In [7]:
def move_to_pregrasp_posotion(dx, dy, dz, k=1):
    robKong.set_tcp((0,0,0.155,0,0,0))
    rospy.sleep(1)
    robKong.translate_tool((0.116 + dx,-0.085 + dy,0),acc=0.05,vel=0.05,wait=True)
    rospy.sleep(2)
    pos = robKong.get_pos()
    print('pos.z',pos.z)
    robKong.translate_tool((0,0,pos.z),acc=0.1,vel=0.1,wait=True)
    rospy.sleep(1)
    if (k == 1):
        arduino_pub2.publish(800)
        rospy.sleep(1)
    elif (k == 2):
        arduino_pub1.publish(800)
        rospy.sleep(1)        
    robKong.translate_tool((0,0,0.078 + dz), acc=0.05, vel=0.05, wait=True)    

In [8]:
rospy.Subscriber(tag_topic1, AprilTagDetectionArray, kong_tag_callback, queue_size=10)

In [17]:
rospy.Subscriber(tag_topic2, AprilTagDetectionArray, hong_tag_callback, queue_size=10)

In [95]:
#kong_step 1: go to home and set the tip of underactuated finger as tcp
robKong_go_to_home_mid()

In [33]:
#kong_step 2*:go to the other hand to make a diagonal crease
robKong.translate((0,-0.2,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.set_tcp((0,-0.063,0.005,0,0,0)) # set the camera as tcp
rospy.sleep(1)
# the following code is to ensure a safe rotation of kong
kong_detect_the_object(2)
rospy.sleep(2)
print (tag_angle1-pi/4)*180/3.1415926

tag_x =  -0.055643590735 ,tag_y =  -0.0561326992932 ,tag_z-  0.253225366624 ,tag angle =  133.630933856
88.6309484018


In [34]:
kong_detect_the_object(2)
rospy.sleep(2)
print (tag_angle1-pi/4)*180/3.1415926

tag_x =  -0.0556923420892 ,tag_y =  -0.0561962241354 ,tag_z-  0.253523056946 ,tag angle =  133.665710441
88.66572499


In [87]:
#Kong_step2: rotate to 45 degree
for i in range(2):
    kong_detect_the_object(2)
    rospy.sleep(2)
    move_ref_tcp((0,0,0,0,0,tag_angle1-pi/4),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
    rospy.sleep(2)

tag_x =  -0.0207885444341 ,tag_y =  0.00055010524145 ,tag_z-  0.218000121181 ,tag angle =  44.8808696896
tag_x =  -0.0209098916063 ,tag_y =  0.000530941565194 ,tag_z-  0.218318974162 ,tag angle =  45.0700498438


In [88]:
#Kong_step 3: approach to the object
kong_approach_the_object(1)
rospy.sleep(1)
while (abs(tag_pose_x1)>=0.001 or abs(tag_pose_y1)>=0.001):
    kong_approach_the_object(1)
    rospy.sleep(3)
    print "1"

tag_x =  -0.0207980834515 ,tag_y =  0.000550263102636 ,tag_z-  0.218831002031 ,tag angle =  44.926810163
1
tag_x =  3.82104692667e-05 ,tag_y =  -0.000561727962991 ,tag_z-  0.218961385477 ,tag angle =  45.0968501863
1


In [89]:
#kong_step 4: move to the pregrasp position, set tcp!!!! 
robKong.set_tcp((0,0,0.155,0,0,0))
rospy.sleep(1)
move_to_pregrasp_posotion(-0.084,0.025,0.004)

('pos.z', 0.021396159268194576)


In [90]:
#kong_step 5:flex-flip and grasp the paper
arduino_pub1.publish(3000)
rospy.sleep(2)
arduino_pub1.publish(1600)
rospy.sleep(3)
robKong.translate_tool((0,0,-0.06),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
arduino_pub1.publish(2800)
arduino_pub2.publish(2800)
rospy.sleep(1)
robKong.translate((0,0,-0.05),acc=0.05,vel=0.05,wait=True)
rospy.sleep(0.5)
robKong.translate_tool((0.04,0,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.71569074,  0.69834708,  0.00990553],
       [ 0.69839266,  0.71571269,  0.00174615],
       [-0.0058701 ,  0.00816765, -0.99994941]])>
<Vector: (-0.02020, -0.81606, -0.07227)>
>

In [92]:
# #kong_step 6: detect the crease and move to the target position
robKong.set_tcp((0,0,0.125,0,0,0)) # set the actuated (2800) fingertip as tcp
rospy.sleep(1)
for i in range(3): 
    kong_detect_the_object(2)
    print('angle',tag_angle1)
    print(tag_pose_y1,tag_pose_x1)
    y=-tag_pose_x1+0.12 # cam to finger:0, tag to target:0.17 - (finger to paper corner)
    x=-(tag_pose_y1+0.056-0.095)
    print("y",y)
    print("x",x)
    rospy.sleep(2)

('angle', 0.7834983070399888)
(0.029920289236122093, -0.017495141597533802)
('y', 0.1414951415975338)
('x', 0.00907971076387791)
('angle', 0.783966255587891)tag_x = 
(0.029942032606239104, -0.017514234876803363) -0.0175142348768 ,tag_y =  
('y', 0.14151423487680337)
('x', 0.009057967393760899)
0.0299420326062 ,tag_z-  0.13112233898 ,tag angle =  44.9179533449
('angle', 0.783966255587891)
(0.029942032606239104, -0.017514234876803363)
('y', 0.14151423487680337)
('x', 0.009057967393760899)
tag_x =  -0.0175123052964 ,tag_y =  0.0299451373918 ,tag_z-  0.131107214757 ,tag angle =  44.9192829571


In [93]:
#kong_step 7: move to the target
robKong.translate_tool((y,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.translate_tool((0,x,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.71587891,  0.69815491,  0.00985446],
       [ 0.69820124,  0.7158989 ,  0.00194889],
       [-0.00569417,  0.00827556, -0.99994954]])>
<Vector: (-0.11559, -0.71072, -0.04320)>
>

In [85]:
robKong.translate_tool((0,x,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.7150667 ,  0.69898548,  0.00994538],
       [ 0.69903207,  0.71508791,  0.00185918],
       [-0.00581228,  0.00828158, -0.99994882]])>
<Vector: (-0.11556, -0.71064, -0.04311)>
>

In [86]:
robKong.translate((0,0,-0.03),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.71498807,  0.69906462,  0.01003583],
       [ 0.69911095,  0.7150112 ,  0.00168917],
       [-0.00599489,  0.0082239 , -0.99994821]])>
<Vector: (-0.11544, -0.71054, -0.07303)>
>

In [94]:
#kong_step 9: rotate and descend
robKong.set_tcp((0,0,0.125,0,0,0)) # set the actuated (2800) fingertip as tcp
rospy.sleep(1)
move_ref_tcp((0,0,0,0,-30*pi/180,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.translate((0,0,-0.01),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.61495381,  0.69834674,  0.36625626],
       [ 0.60562626,  0.71571568, -0.3478044 ],
       [-0.50502341,  0.00793077, -0.86306921]])>
<Vector: (-0.11550, -0.71062, -0.05300)>
>

In [32]:
#hong_step 5: move out 
robHong.translate((0,0,0.05),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((-0.05,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0,0.05,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-5.27677053e-02, -9.98599900e-01, -3.71596247e-03],
       [-9.98606796e-01,  5.27680563e-02,  3.60474810e-06],
       [ 1.92484416e-04,  3.71097559e-03, -9.99993096e-01]])>
<Vector: (0.24993, -0.88538, -0.04503)>
>

In [35]:
#hong_step 6: detect the tag and rotate roller to make a crease
hong_detect_the_object(2)
rospy.sleep(2)
print (tag_angle+pi/2)*180/3.1415926
robHong.set_tcp((0.15,0,0,0,0,0)) #set the camera as tcp
rospy.sleep(1)
print (tag_angle+pi/2+pi/4)*180/3.1415926
for i in range(2):
    hong_detect_the_object(2)
    rospy.sleep(2)
    move_ref_tcp_hong((0,0,0,0,0,tag_angle+pi/2+pi/4),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True) #move_angle is related to the hold orientation
    rospy.sleep(2)

tag_x =  9.68604072468e-06 ,tag_y =  6.09904635857e-05 ,tag_z-  0.289764944568 ,tag angle =  -90.0375487507
-0.037557532977
44.9624432346
tag_x =  1.55605431056e-06 ,tag_y =  -2.54794087639e-05 ,tag_z-  0.289652429627 ,tag angle =  -90.1300997975
tag_x =  -0.0113360406474 ,tag_y =  -0.00414918817571 ,tag_z-  0.290851809495 ,tag angle =  -135.158383601


In [36]:
#hong_step 6: approach the tag
hong_approach_the_object(1)
rospy.sleep(2)
while (abs(tag_pose_x)>0.001 or abs(tag_pose_y)>0.001):
    hong_approach_the_object(1)
    rospy.sleep(2)

tag_x =  -0.0112728563569 ,tag_y =  -0.00410594262355 ,tag_z-  0.291228020051 ,tag angle =  -135.015126005
tag_x =  0.000100173996948 ,tag_y =  0.000631795335124 ,tag_z-  0.290495697106 ,tag angle =  -135.036470026


In [37]:
# hong_step 7: set the roller tip as tcp, and rotate to 45 (crease angle is 45) with new tcp, and rotate 20 because holeder finger is taller than roller
robHong.set_tcp((-0.07,0,0.315,0,0,0))
rospy.sleep(1)
move_ref_tcp_hong((0,0,0,0,pi*20/180,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.set_tcp((-0.07,0,0.315,0,-pi*20/180,0))
rospy.sleep(1)

In [39]:
#hong_step 8: move to the target position for apply a crease
# robHong.translate((-0.018,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate((0,-0.28,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate((0,0,-0.055),acc=0.05,vel=0.05,wait=True)
# robHong.translate_tool((0,0.07,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate_tool((0.5,0,0),acc=0.05,vel=0.05,wait=True)
robHong.set_tcp((-0.07,0,0.315,0,-pi*20/180,0))
rospy.sleep(1)
pos = robHong.get_pos()
print pos.z
robHong.translate((0,0,-pos.z),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate((0,0,-0.03),acc=0.05,vel=0.05,wait=True)

-0.03779273062298161


<Transform:
<Orientation: 
array([[-7.39958795e-01, -6.72641136e-01, -3.85797068e-03],
       [-6.72646586e-01,  7.39963881e-01,  1.58332081e-04],
       [ 2.74825829e-03,  2.71221002e-03, -9.99992545e-01]])>
<Vector: (0.40033, -0.77970, -0.02977)>
>

In [43]:
#hong_step 9: move to the target position for apply a crease
robHong.set_tcp((-0.07,0,0.315,0,-pi*20/180,0))
rospy.sleep(1)
# robHong.translate_tool((0,0.1,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate_tool((0.5,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
robHong.translate((0,0,-0.067),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-7.39890561e-01, -6.72715957e-01, -3.89871430e-03],
       [-6.72722708e-01,  7.39894441e-01,  6.11644996e-04],
       [ 2.47317369e-03,  3.07530400e-03, -9.99992213e-01]])>
<Vector: (-0.02743, -1.05245, -0.09549)>
>

In [51]:
#hong_step 10: apply a crease
robHong.set_tcp((-0.07,0,0.315,0,-pi*20/180,0))
rospy.sleep(1)
robHong.translate_tool((-0.07,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0.07,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((-0.07,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0.07,0,0),acc=0.05,vel=0.05,wait=True)

In [58]:
robHong.translate((0,0,0.1),acc=0.05,vel=0.05,wait=True)
robHong.translate_tool((-0.15,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate((0,0,-0.1),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((-0.02,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0.06,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((-0.06,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)

In [63]:
#hong_step 11:move out hong and kong
# robHong.translate_tool((0,0,-0.1),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate_tool((0,0.1,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robKong.translate_tool((0,0,-0.1),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# arduino_pub1.publish(0)
# rospy.sleep(0.5)
# arduino_pub2.publish(0)
robHong_go_to_home_mid()
rospy.sleep(1)
robHong.set_tcp((0.075,0,0.323,0,0,0))
rospy.sleep(1)
robKong_go_to_home_mid()

In [64]:
#kong_step 10:move to the other side
robKong.translate((-0.3,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)

In [69]:
#hong_step 12:press the other side
robHong.set_tcp((0.075,0,0.323,0,0,0))
rospy.sleep(1)
move_ref_tcp_hong((0,0,0,0,0,pi),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((-0.17,-0.23,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
move_ref_tcp_hong((0,0,0,pi/180,0,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
pos = robHong.get_pos()
print('pos.z',pos.z)
robHong.translate_tool((0,0,pos.z),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0,0,0.095), acc=0.05, vel=0.05, wait=True) 

('pos.z', -6.396906253786083e-05)


<Transform:
<Orientation: 
array([[ 2.24096754e-02,  9.98998334e-01, -3.87315715e-02],
       [ 9.99748848e-01, -2.24013523e-02,  6.48915190e-04],
       [-2.19374385e-04, -3.87363859e-02, -9.99249440e-01]])>
<Vector: (-0.06033, -1.01734, -0.09467)>
>

In [70]:
#kong_step 11*:go to the other hand to make a diagonal crease
robKong.translate((0,-0.2,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.set_tcp((0,-0.063,0.005,0,0,0)) # set the camera as tcp
rospy.sleep(1)
# the following code is to ensure a safe rotation of kong
kong_detect_the_object(2)
rospy.sleep(2)
print (tag_angle1)*180/3.1415926

tag_x =  -0.0291062476957 ,tag_y =  -0.0258734843176 ,tag_z-  0.262661324286 ,tag angle =  134.752054489
134.752069931


In [26]:
print (tag_angle1-pi*3/4)*180/3.1415926

-0.686420612039


In [71]:
#Kong_step 11: rotate to 45 degree
for i in range(2):
    kong_detect_the_object(2)
    rospy.sleep(2)
    move_ref_tcp((0,0,0,0,0,tag_angle1-pi*3/4),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
    rospy.sleep(2)

tag_x =  -0.0291388509956 ,tag_y =  -0.0259032601341 ,tag_z-  0.262910872311 ,tag angle =  134.833454145
tag_x =  -0.0290239403788 ,tag_y =  -0.0259941540026 ,tag_z-  0.262429129175 ,tag angle =  135.048440722


In [78]:
#Kong_step 12: approach to the object
kong_approach_the_object(1)
rospy.sleep(1)
while (abs(tag_pose_x1)>=0.001 or abs(tag_pose_y1)>=0.001):
    kong_approach_the_object(1)
    rospy.sleep(3)
    print "1"

tag_x =  -0.0535949067583 ,tag_y =  -0.0592109590147 ,tag_z-  0.251797244807 ,tag angle =  134.768757003
1
1
tag_x =  0.00166624545711 ,tag_y =  -0.00058696007928 ,tag_z-  0.253630377005 ,tag angle =  135.14705724
1
tag_x =  -1.48058606425e-06 ,tag_y =  2.57326528379e-05 ,tag_z-  0.253500807153 ,tag angle =  135.168091219
1


In [79]:
#kong_step 13: move to the pregrasp position!!!! 
robKong.set_tcp((0,0,0.155,0,0,0))
rospy.sleep(1)
move_to_pregrasp_posotion(-0.162,0.025,0.002, 2)
# robKong.translate_tool((-0.056,-0.06,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(2)
# pos = robKong.get_pos()
# print('pos.z',pos.z)

('pos.z', 0.049412964289423794)


In [80]:
#kong_step 14:flex-flip and grasp the paper
arduino_pub2.publish(3000)
rospy.sleep(2)
arduino_pub2.publish(1600)
rospy.sleep(3)
robKong.translate_tool((0,0,-0.04),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
arduino_pub2.publish(2200)
rospy.sleep(0.5)
arduino_pub1.publish(2200)

In [81]:
#kong_step 15: move to the target
# robKong.translate_tool((-y,0,0),acc=0.05,vel=0.05,wait=True)
robKong.translate_tool((-0.06,0,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.70306616, -0.71104072,  0.01091172],
       [-0.71105706,  0.70312801,  0.00297789],
       [-0.00978973, -0.0056652 , -0.99993603]])>
<Vector: (-0.15964, -0.80382, -0.03966)>
>

In [83]:
#kong_step 9: rotate and descend
move_ref_tcp((0,0,0,0,50*pi/180,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.translate((0,0,-0.015),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
arduino_pub1.publish(1800)
rospy.sleep(0.5)
arduino_pub2.publish(1800)

In [84]:
#hong_step 5: move out 
robHong.translate((0,0,0.05),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0,-0.05,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0.2,0,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 2.24795431e-02,  9.98994454e-01, -3.87911094e-02],
       [ 9.99747282e-01, -2.24545543e-02,  1.07980631e-03],
       [ 2.07683440e-04, -3.88055797e-02, -9.99246758e-01]])>
<Vector: (-0.10595, -0.81619, -0.04322)>
>

In [86]:
# hong_step 12: set the roller tip as tcp, and rotate to 45 (crease angle is 45) with new tcp, and rotate 20 because holeder finger is taller than roller
robHong.set_tcp((-0.07,0,0.315,0,0,0))
rospy.sleep(1)
# move_ref_tcp_hong((0,0,0,0,0,-pi*45/180),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# move_ref_tcp_hong((0,0,0,0,pi*20/180,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.set_tcp((-0.07,0,0.315,0,-pi*20/180,0))
# rospy.sleep(1)
pos = robHong.get_pos()
print pos.z
robHong.translate((0,0,-pos.z),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate((0,0,-0.03),acc=0.05,vel=0.05,wait=True)

-0.03520962485507745


<Transform:
<Orientation: 
array([[-0.63557606,  0.7223507 , -0.2724932 ],
       [ 0.67894965,  0.69098744,  0.2481204 ],
       [ 0.36751932, -0.02730978, -0.92961483]])>
<Vector: (-0.10885, -0.96119, -0.02974)>
>

In [90]:
#hong_step 13: move to the target position for apply a crease
robHong.set_tcp((-0.07,0,0.315,0,-pi*20/180,0))
rospy.sleep(1)
robHong.translate_tool((0,0.19,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((-0.12,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate((0,0,-0.056),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.69059028,  0.72220789, -0.03874044],
       [ 0.72271219,  0.69114769,  0.00140158],
       [ 0.0277876 , -0.02703027, -0.99924832]])>
<Vector: (0.11116, -0.91645, -0.09453)>
>

In [93]:
#hong_step 14: apply a crease
robHong.set_tcp((-0.07,0,0.315,0,-pi*20/180,0))
rospy.sleep(1)
robHong.translate_tool((-0.05,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate_tool((0.05,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
# robHong.translate_tool((0.25,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate_tool((-0.25,0,0),acc=0.05,vel=0.05,wait=True)

In [98]:
# robHong.translate((0,0,0.1),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate_tool((-0.15,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate((0,0,-0.1),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate_tool((-0.02,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate_tool((0.06,0,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
robHong.translate_tool((-0.06,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)

In [104]:
# robHong.translate_tool((0,0,-0.1),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robHong.translate((0,0.2,0),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
# robKong.translate((0,0,0.1),acc=0.05,vel=0.05,wait=True)
# rospy.sleep(1)
arduino_pub1.publish(0)
rospy.sleep(0.5)
arduino_pub2.publish(0)
robHong_go_to_home_mid()
rospy.sleep(1)
robHong.set_tcp((0.075,0,0.323,0,0,0))
rospy.sleep(1)
robKong_go_to_home_mid()